In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
# import pytorch_DIW_scratch
import torch.nn as nn
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
# from blurgenerator import motion_blur, lens_blur, gaussian_blur
from PIL import Image
import torchvision.transforms as T
import pandas as pd
import PIL.Image as pil
from torchvision import transforms, datasets

top = T.ToPILImage()
tot = T.ToTensor()

import cv2
import os

from tqdm import tqdm
import sys

# sys.path.insert(1,'PerceptualSimilarity')
# import models
import unet_parts as util
import lpips

In [2]:
""" Full assembly of the parts to form the complete network """

from unet_parts import *


class UNet(nn.Module):
    def __init__(self, n_channels_org, n_channels_depth, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels_org = n_channels_org
        self.n_channels_depth = n_channels_depth
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc_org = (DoubleConv(n_channels_org, 32))
        self.inc_depth = (DoubleConv(n_channels_depth, 32))

        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))

        self.outc_0 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False), # 3, 32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False), # 3, 32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.outc = (OutConv(64, n_classes))

    def forward(self, depth, org):
        d1 = self.inc_depth(depth)
        org1 = self.inc_org(org)

        x1 = torch.cat([d1, org1], axis=1) # (b, 64, w, h)

        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc_0(x)
        x = self.outc(x)

        x = F.softmax(x, dim=1)
        return x

model = UNet(3, 1, 4).cuda()


# model_parameters = filter(lambda p: p.requires_grad, model.parameters())
# sum([np.prod(p.size()) for p in model_parameters])

# 4125956

In [3]:
# https://github.com/Soester10/Bokeh-Rendering-with-Vision-Transformers/blob/main/train.py

import PIL.Image as pil
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# feed_width = 768
# feed_height =  512
# feed_height = 384
# feed_width = 512
# feed_height = 768
# feed_width = 1024

feed_height = 1024
feed_width = 1536

batch_size = 1#0


class bokehDataset(Dataset):
    
    def __init__(self, csv_file,root_dir, transform=None):
        
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        bok = pil.open(self.root_dir + self.data.iloc[idx, 0][1:]).convert('RGB')
        org = pil.open(self.root_dir + self.data.iloc[idx, 1][1:]).convert('RGB')
        original_width, original_height = org.size

        depth_path = str(self.root_dir + self.data.iloc[idx, 1][1:]
                         ).replace('original', 'depth').replace('jpg', 'png')
        depth = pil.open(depth_path)

        blur_6_path = str(self.root_dir + self.data.iloc[idx, 0][1:]).replace('bokeh', 'lens_blur_6_3_2')
        blur_17_path = str(self.root_dir + self.data.iloc[idx, 0][1:]).replace('bokeh', 'lens_blur_17_3_2')
        blur_65_path = str(self.root_dir + self.data.iloc[idx, 0][1:]).replace('bokeh', 'lens_blur_65_3_3')
        blur_6_path = pil.open(blur_6_path).convert('RGB')
        blur_17_path = pil.open(blur_17_path).convert('RGB')
        blur_65_path = pil.open(blur_65_path).convert('RGB')

        bok = bok.resize((feed_width, feed_height), pil.LANCZOS)
        org = org.resize((feed_width, feed_height), pil.LANCZOS)
        depth = depth.resize((feed_width, feed_height), pil.LANCZOS)

        blur_6_path = blur_6_path.resize((feed_width, feed_height), pil.LANCZOS)
        blur_17_path = blur_17_path.resize((feed_width, feed_height), pil.LANCZOS)
        blur_65_path = blur_65_path.resize((feed_width, feed_height), pil.LANCZOS)


        if self.transform : 
            bok_dep = self.transform(bok)
            org_dep = self.transform(org)
            depth_dep = self.transform(depth)

            blur_6_path = self.transform(blur_6_path)

            blur_17_path = self.transform(blur_17_path)

            blur_65_path = self.transform(blur_65_path)
        
        stacked_10 = torch.stack([org_dep, blur_6_path, blur_17_path, blur_65_path], dim=0)
        return (bok_dep, org_dep, depth_dep, stacked_10, self.root_dir + self.data.iloc[idx, 1][1:], 
                original_width, original_height) # here

transform1 = transforms.Compose([
    transforms.ToTensor(),
])

testset = bokehDataset(csv_file = '../MegaDepth/Bokeh_Data/test.csv',  root_dir = '.', transform = transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=7)


In [4]:
import lpips

device = 'cuda'
                                                  
MSE_LossFn = nn.MSELoss()
L1_LossFn = nn.L1Loss()
lpips_fn = lpips.LPIPS(net='alex')#.cuda() # models.PerceptualLoss(model='net-lin',net='alex',use_gpu=opt.use_gpu)

parameters_path = '/home/ubuntu/max/image/JPEG/C++_pipeline/zip/depth_estimation/DPT/weights/top_4_blurs_finetune/55.pt'
parameters = torch.load(parameters_path, map_location=torch.device("cpu"))
model.load_state_dict(parameters)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/ubuntu/max/image/JPEG/C++_pipeline/zip/depth_estimation/DPT/PerceptualSimilarity/lpips/weights/v0.1/alex.pth


<All keys matched successfully>

In [5]:
with torch.no_grad():
    lpips_fn = lpips_fn.cuda()
    running_l1_loss = 0
    running_ms_loss = 0
    running_lpips_loss = 0

    for i,data in enumerate(tqdm(testloader), 0):
        device = 'cuda'
        bok, org, depth, stacked_10, path, original_width, original_height = data
        bok, org, depth = bok.to(device) , org.to(device), depth.to(device, dtype=torch.float)
        stacked_10 = stacked_10.to(device)

        # bok_mask = model.forward(depth)
        bok_mask = model.forward(depth, org)

        # stacked_images = torch.stack([org, blur_25, blur_45, blur_75], dim=1).cuda()  # [batch_n, 4, 3, h, w]
        # stacked_images = torch.stack([org, blur_45], dim=1).cuda()  # [batch_n, 4, 3, h, w]


        # Multiply the maps and images using broadcasting
        bok_mask = bok_mask.unsqueeze(2)
        output_images = torch.mul(bok_mask, stacked_10) # [batch_n, 4, 3, h, w]
        bok_pred = torch.sum(output_images, dim=1) # [batch_n, 3, h, w]


        
        l1_loss = L1_LossFn(bok_pred, bok)
        ms_loss = 1-ssim(bok_pred, bok)
        lpips_loss = lpips_fn.forward(bok_pred, bok)

        running_l1_loss += l1_loss.item()
        running_ms_loss += ms_loss.item()
        running_lpips_loss += lpips_loss.item()


        # top(bok_pred[0]).save(path)
        # print('before', bok_pred.shape)
        bok_pred = F.interpolate(bok_pred,(original_height,original_width),mode = 'bilinear')

        path = path[0].replace('original', 'val294_55')
        # path_bok = path.replace('val294', 'bok294')

        top(bok_pred[0]).save(path)

100%|██████████| 294/294 [01:15<00:00,  3.91it/s]


In [10]:
running_l1_loss / 294, running_ms_loss, running_lpips_loss / 294
# # (0.04888411183670467, 0.21529191732406616, 0.10059311754089229) 42
# # (0.0483693949003913,  0.2130470871925354,   0.09418721285452243)   45


In [1]:
dir0 = '/Users/max/Documents/diploma/server/variable_blur_radius/4_power'
# dir0 = '/home/ubuntu/max/image/JPEG/C++_pipeline/zip/DMSHN/val294'
dir1 = '/Users/max/Documents/diploma/server/val/val294_bok'
# dir1 = '/home/ubuntu/max/image/JPEG/C++_pipeline/zip/DMSHN/bok294_dmshn'

## Initializing the model
import lpips
model = lpips.LPIPS(net='alex') # models.PerceptualLoss(model='net-lin',net='alex',use_gpu=opt.use_gpu)
model = model#.cuda()
import os
import cv2
from tqdm import tqdm
import unet_parts as util
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# crawl directories
files = os.listdir(dir0)

total_dist = 0
total_psnr = 0
total_ssim = 0
count =0

for file in tqdm(files):
	file1 = file[:4] + '.jpg'
	if(os.path.exists(os.path.join(dir1,file1))):
		# Load images
		img0 = util.im2tensor(util.load_image(os.path.join(dir0,file))) # RGB image from [-1,1]
		img1 = util.im2tensor(util.load_image(os.path.join(dir1,file1)))
		# img0 = img0.cuda()
		# img1 = img1.cuda()

		# Compute distance
		dist01 = model.forward(img0,img1)
		total_dist += dist01.item()

		I0 = cv2.imread(os.path.join(dir0,file))
		I1 = cv2.imread(os.path.join(dir1,file1))

		total_psnr += compare_psnr(I0,I1)
		total_ssim += compare_ssim(I0,I1,multichannel=True, channel_axis=2)

		count +=1

print ('Avg LPIPS: ', total_dist/len(files))
print ('Avg PSNR: ', total_psnr/len(files))
print ('Avg SSIM: ', total_ssim/len(files))
print ('Total files: ', count)
print ("Weights:", parameters_path)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/Users/max/opt/anaconda3/envs/max_2_0/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/max/opt/anaconda3/envs/max_2_0/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/max/opt/anaconda3/envs/max_2_0/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


  0%|          | 0/294 [00:00<?, ?it/s]/var/folders/7b/dxvk6ctn7yzgkzt7ccsrrtgh0000gn/T/ipykernel_86629/3166847299.py:42: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  total_ssim += compare_ssim(I0,I1,multichannel=True, channel_axis=2)
100%|██████████| 294/294 [06:22<00:00,  1.30s/it]


Avg LPIPS:  0.1880474775193297
Avg PSNR:  24.019705778397935
Avg SSIM:  0.8638111441949295
Total files:  294


NameError: name 'parameters_path' is not defined

In [9]:
# 15
# Avg LPIPS:  0.25091465978192634
# Avg PSNR:  23.932834724123268
# Avg SSIM:  0.8544052056915824

# 23
# Avg LPIPS:  0.24972314648583632
# Avg PSNR:  24.100258935649894
# Avg SSIM:  0.8538297881025634

# 40
# Avg LPIPS:  0.2051788743357269
# Avg PSNR:  23.915384389011518
# Avg SSIM:  0.8607180569365682

# 50
# Avg LPIPS:  0.1780613246468865
# Avg PSNR:  24.244362704694105
# Avg SSIM:  0.8666627774577712

# 55
# Avg LPIPS:  0.17861873519663907
# Avg PSNR:  24.26308359282466
# Avg SSIM:  0.8672498382263079

# 56
# Avg LPIPS:  0.17850339405086577
# Avg PSNR:  24.262182775017312
# Avg SSIM:  0.8669904421732371



In [4]:
import os

def delete_images_except(directory, keep_filenames):
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg") and filename not in keep_filenames:
                os.remove(os.path.join(root, filename))

# Example usage
directory_path = "/Users/max/Documents/diploma/server/variable_blur_radius"
keep_filenames = ["4507.jpg", "4682.jpg"]

delete_images_except(directory_path, keep_filenames)


In [ ]:
# Mine
# Avg LPIPS:  0.17618930433261193
# Avg PSNR:  24.200597489832337
# Avg SSIM:  0.8668268319166299

# 4 power

# BGGAN
# Avg LPIPS:  0.1473597307138297
# Avg PSNR:  25.969951218691634
# Avg SSIM:  0.8789945434643803



# DMSHN
# Avg LPIPS:  0.22179737745397757
# Avg PSNR:  24.631763065227975
# Avg SSIM:  0.8703852476185571

# SDMSHN
# Avg LPIPS:  0.21703358799168448
# Avg PSNR:  24.700831853744432
# Avg SSIM:  0.873192968526886



In [ ]:
# Epoch 50 finetune lpips + ssim + L1 interpolate
# Avg LPIPS:  0.1780613246468865
# Avg PSNR:  24.244362704694105
# Avg SSIM:  0.8666627774577712

# Epoch 45 finetune lpips + ssim + L1 interpolate
# Avg LPIPS:  0.18527495844581096
# Avg PSNR:  24.15010095669144
# Avg SSIM:  0.8644536434417619


# Epoch 42 finetune lpips + ssim + L1 interpolate
# Avg LPIPS:  0.20383347660860643
# Avg PSNR:  23.9200654617216
# Avg SSIM:  0.8610615226189505


# Stacked DMSHN
# Avg LPIPS:  0.2529120883067786
# Avg PSNR:  28.237889625140056
# Avg SSIM:  0.915850495268047




# Epoch 42 finetune lpips + ssim + L1
# Avg LPIPS:  0.18566639696051473
# Avg PSNR:  23.916317089155488
# Avg SSIM:  0.8598279403285225


# Epoch 30 finetune lpips + ssim + L1
# Avg LPIPS:  0.19235397890514258
# Avg PSNR:  23.758934071366454
# Avg SSIM:  0.8574985115548894



# Epoch 26 finetune lpips
# Avg LPIPS:  0.20650175170732193
# Avg PSNR:  23.685084036762046
# Avg SSIM:  0.8530066391542362


# Last epoch 22
# Avg LPIPS:  0.23184171818247457
# Avg PSNR:  23.995127629918684
# Avg SSIM:  0.8532735757485003


# Last epoch 23/24
# Avg LPIPS:  0.22942917664744417
# Avg PSNR:  24.092180066127206
# Avg SSIM:  0.8524654714009784

# Last epoch 27
# Avg LPIPS:  0.2635695703175603
# Avg PSNR:  24.05714959195201
# Avg SSIM:  0.8466768354282206


# Last epoch 29
# Avg LPIPS:  0.27070816129851505
# Avg PSNR:  24.08397440610926
# Avg SSIM:  0.8443834677619755

# Last epoch 30
# Avg LPIPS:  0.26866581653352495
# Avg PSNR:  24.107443187852514
# Avg SSIM:  0.8456310957087362


# Last epoch 32
# Avg LPIPS:  0.28595100530759004
# Avg PSNR:  24.050488337605206
# Avg SSIM:  0.8419934323655651

In [ ]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

dir0 = 'Training/val294'
# dir0 = '/home/ubuntu/max/image/JPEG/C++_pipeline/zip/DMSHN/val294'
dir1 = 'Training/val294_bok'

# Initialize the model
model = lpips.LPIPS(net='alex')
model = model#.cuda()

# Crawl directories
files = os.listdir(dir0)

total_dist = 0
total_psnr = 0
total_ssim = 0
count = 0


def process_file(file):
    file1 = file[:4] + '.jpg'
    if os.path.exists(os.path.join(dir1, file1)):
        # Load images
        img0 = util.im2tensor(util.load_image(os.path.join(dir0, file)))  # RGB image from [-1, 1]
        img1 = util.im2tensor(util.load_image(os.path.join(dir1, file1)))

        # img0 = img0.cuda()
        # img1 = img1.cuda()

        # Compute distance
        dist01 = model.forward(img0, img1)

        I0 = cv2.imread(os.path.join(dir0, file))
        I1 = cv2.imread(os.path.join(dir1, file1))

        psnr = compare_psnr(I0, I1)
        ssim = compare_ssim(I0, I1, multichannel=True, channel_axis=2)

        return dist01.item(), psnr, ssim, 1
    return 0, 0, 0, 0


with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_file, files), total=len(files)))

for dist, psnr, ssim, valid in results:
    total_dist += dist
    total_psnr += psnr
    total_ssim += ssim
    count += valid

print('Avg LPIPS: ', total_dist / count)
print('Avg PSNR: ', total_psnr / count)
print('Avg SSIM: ', total_ssim / count)
print('Total files: ', count)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/ubuntu/anaconda3/envs/max_2_0/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


  5%|▍         | 14/294 [00:20<06:53,  1.48s/it]


KeyboardInterrupt: 

In [ ]:
# Avg LPIPS:  0.24802155055257738
# Avg PSNR:  24.237881301361597
# Avg SSIM:  0.850600642181404
